# Interactive Dashboard App

**Project Goal:** To build a dynamic web application that will allow users to choose their own features, build a model, and evaluate its performance through a graphic user interface - a tool that will allow anyone to build a model without code.

**Specific Goals:**
- Import and clean dataset
- Build web application layout
- Build interactive elements
- Launch Application.


Build Application:
- Application Layout
- Variance bar chart: Call back, decorator
- K-Means slider
- PCA scatter plot

In [26]:
import pandas as pd 
import plotly.express as px
from dash import Input, Output, dcc, html
from jupyter_dash import JupyterDash
from scipy.stats.mstats import trimmed_var 
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

### Prepare Data

Import

- Creating a wrangle function to import the data:

In [13]:
def wrangle(filepath):

    """Read SCF data file into ``DataFrame``.

    Returns only credit fearful households whose net worth is less than $2 million.

    Parameters
    ----------
    filepath : str
        Location of CSV file.
    """
    df = pd.read_csv(filepath)
    
    mask = (df["TURNFEAR"]==1) & (df["NETWORTH"]< 2e6)
    df = df[mask]
    
    return df

- Using the wrangle function to import the data:

In [18]:
df=wrangle("data/customer.csv")
print("df type:", type(df))
print("df shape:", df.shape)
df.head()

df type: <class 'pandas.core.frame.DataFrame'>
df shape: (4418, 351)


,YY1,Y1,WGT,HHSEX,AGE,AGECL,EDUC,EDCL,MARRIED,KIDS,...,NWCAT,INCCAT,ASSETCAT,NINCCAT,NINC2CAT,NWPCTLECAT,INCPCTLECAT,NINCPCTLECAT,INCQRTCAT,NINCQRTCAT
5,2,21,3790.476607,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2
6,2,22,3798.868505,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,3,2,2
7,2,23,3799.468393,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2
8,2,24,3788.076005,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2
9,2,25,3793.066589,1,50,3,8,2,1,3,...,1,2,1,2,1,1,4,4,2,2


### Build Dashboard

#### Application Layout

Instatntiate Application

In [24]:
app = JupyterDash(__name__)

print("app type:", type(app))

app type: <class 'jupyter_dash.jupyter_app.JupyterDash'>


Layout

In [ ]:
app.layout = 

### Application Deployment

In [27]:
app.run_server(host="0.0.0.0", mode="external")

ConnectionError: HTTPConnectionPool(host='0.0.0.0', port=8050): Max retries exceeded with url: /_alive_9c147a48-d5a6-4d24-aca9-d9e04e0d1f15 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x000001A65F21CAD0>: Failed to establish a new connection: [WinError 10049] The requested address is not valid in its context'))